# Enhanced Fingerprint Matching System with Coarse Indexing

This notebook implements a high-performance fingerprint identification system with coarse indexing for large-scale databases.

## Features:
- **Global Ridge Flow Classification**: Groups fingerprints by dominant orientation patterns
- **Minutiae Distribution Clustering**: Spatial distribution and density-based clustering
- **Local Hash Buckets**: Hash-based indexing of local minutiae arrangements
- **Geometric Clustering**: K-means clustering on feature vectors
- **10-50x Speed Improvement** over exhaustive search

In [14]:
# Import required libraries
import os
import cv2
import math
import fingerprint_enhancer
import numpy as np
from pymongo import MongoClient
from fingerprint_feature_extractor import extract_minutiae_features
from scipy.optimize import linear_sum_assignment
from collections import defaultdict
import hashlib
from sklearn.cluster import KMeans
import pickle

print("Libraries imported successfully!")

Libraries imported successfully!


In [15]:
# Database connection
client = MongoClient('mongodb://localhost:27017/')
db = client['fingerprint_db']
collection = db['features']
index_collection = db['coarse_index']  # New collection for indexing

print("Database connection established")
print(f"Total fingerprints in database: {collection.count_documents({})}")

Database connection established
Total fingerprints in database: 803


In [16]:
class FingerprintCoarseIndex:
    """
    Coarse indexing system for large-scale fingerprint identification
    Uses multiple indexing strategies:
    1. Global ridge flow classification
    2. Minutiae distribution patterns
    3. Local minutiae arrangements (hash buckets)
    4. Geometric clustering
    """
    
    def __init__(self):
        self.ridge_flow_clusters = None
        self.minutiae_distribution_clusters = None
        self.hash_table = defaultdict(list)
        self.index_built = False
    
    def extract_global_features(self, minutiae_data):
        """
        Extract global features for coarse indexing
        
        Returns:
        --------
        dict: Global features including ridge flow, distribution patterns, etc.
        """
        if not minutiae_data:
            return None
            
        terminations = minutiae_data.get('terminations', [])
        bifurcations = minutiae_data.get('bifurcations', [])
        all_minutiae = terminations + bifurcations
        
        if len(all_minutiae) < 3:
            return None
            
        features = {}
        
        # 1. Basic statistics
        features['total_minutiae'] = len(all_minutiae)
        features['termination_ratio'] = len(terminations) / len(all_minutiae) if all_minutiae else 0
        
        # 2. Spatial distribution features
        x_coords = [m['x'] for m in all_minutiae]
        y_coords = [m['y'] for m in all_minutiae]
        
        features['x_mean'] = np.mean(x_coords)
        features['y_mean'] = np.mean(y_coords)
        features['x_std'] = np.std(x_coords)
        features['y_std'] = np.std(y_coords)
        
        # 3. Orientation distribution (global ridge flow approximation)
        angles = []
        for m in all_minutiae:
            if m['angle'] and len(m['angle']) > 0 and m['angle'][0] is not None:
                angles.append(m['angle'][0])
        
        if angles:
            # Convert to unit vectors and compute dominant direction
            cos_sum = sum(np.cos(2 * a) for a in angles)  # Double angle for orientation
            sin_sum = sum(np.sin(2 * a) for a in angles)
            features['dominant_orientation'] = np.arctan2(sin_sum, cos_sum) / 2
            features['orientation_coherence'] = np.sqrt(cos_sum**2 + sin_sum**2) / len(angles)
        else:
            features['dominant_orientation'] = 0
            features['orientation_coherence'] = 0
            
        # 4. Minutiae density in different regions (divide image into 4 quadrants)
        if x_coords and y_coords:
            x_mid, y_mid = np.median(x_coords), np.median(y_coords)
            quadrants = [0, 0, 0, 0]  # top-left, top-right, bottom-left, bottom-right
            
            for x, y in zip(x_coords, y_coords):
                if x <= x_mid and y <= y_mid:
                    quadrants[0] += 1
                elif x > x_mid and y <= y_mid:
                    quadrants[1] += 1
                elif x <= x_mid and y > y_mid:
                    quadrants[2] += 1
                else:
                    quadrants[3] += 1
            
            # Normalize by total minutiae
            features['quad_density'] = [q / len(all_minutiae) for q in quadrants]
        else:
            features['quad_density'] = [0.25, 0.25, 0.25, 0.25]
            
        return features
    
    def compute_minutiae_hash(self, minutiae_data, hash_size=1000):
        """
        Compute hash buckets based on local minutiae arrangements
        """
        terminations = minutiae_data.get('terminations', [])
        bifurcations = minutiae_data.get('bifurcations', [])
        all_minutiae = terminations + bifurcations
        
        if len(all_minutiae) < 3:
            return []
            
        hashes = []
        
        # For each minutia, create local arrangement descriptor
        for i, center_minutia in enumerate(all_minutiae):
            # Find nearest neighbors
            distances = []
            for j, other_minutia in enumerate(all_minutiae):
                if i != j:
                    dx = other_minutia['x'] - center_minutia['x']
                    dy = other_minutia['y'] - center_minutia['y']
                    dist = np.sqrt(dx*dx + dy*dy)
                    distances.append((dist, j))
            
            # Sort by distance and take closest neighbors
            distances.sort()
            n_neighbors = min(4, len(distances))  # Use up to 4 nearest neighbors
            
            if n_neighbors >= 2:
                # Create invariant descriptor
                descriptor = []
                center_angle = center_minutia['angle'][0] if center_minutia['angle'] and center_minutia['angle'][0] else 0
                
                for k in range(n_neighbors):
                    _, neighbor_idx = distances[k]
                    neighbor = all_minutiae[neighbor_idx]
                    
                    # Relative position (translation invariant)
                    dx = neighbor['x'] - center_minutia['x']
                    dy = neighbor['y'] - center_minutia['y']
                    
                    # Make rotation invariant by normalizing with center minutia angle
                    rel_angle = np.arctan2(dy, dx) - center_angle
                    rel_dist = np.sqrt(dx*dx + dy*dy)
                    
                    # Quantize for robustness
                    angle_bin = int((rel_angle % (2*np.pi)) * 8 / (2*np.pi))  # 8 angle bins
                    dist_bin = min(int(rel_dist / 10), 9)  # 10 distance bins
                    
                    descriptor.extend([angle_bin, dist_bin])
                
                # Convert to hash
                hash_str = ''.join(map(str, descriptor))
                hash_val = int(hashlib.md5(hash_str.encode()).hexdigest(), 16) % hash_size
                hashes.append(hash_val)
        
        return hashes
    

    def build_index(self):
        """
        Build coarse index from all fingerprints in database
        """
        print("🔨 Building coarse index...")
        
        # Clear existing index
        index_collection.delete_many({})
        
        # Extract features from all fingerprints
        all_features = []
        fingerprint_ids = []
        
        for record in collection.find():
            features = self.extract_global_features(record)
            if features:
                all_features.append([
                    features['total_minutiae'],
                    features['termination_ratio'],
                    features['x_std'],
                    features['y_std'],
                    features['dominant_orientation'],
                    features['orientation_coherence']
                ] + features['quad_density'])
                
                fingerprint_ids.append(record['_id'])
                
                # Store in index collection
                index_record = {
                    'fingerprint_id': record['_id'],
                    'filename': record['filename'],
                    'global_features': features,
                    'hash_buckets': self.compute_minutiae_hash(record)
                }
                index_collection.insert_one(index_record)
        
        # Build clustering models
        if len(all_features) > 10:
            # Ridge flow clustering
            n_ridge_clusters = min(10, len(all_features) // 50 + 1)
            self.ridge_flow_clusters = KMeans(n_clusters=n_ridge_clusters, random_state=42, n_init=10)
            ridge_features = [[f[4], f[5]] for f in all_features]
            self.ridge_flow_clusters.fit(ridge_features)
            
            # Minutiae distribution clustering
            n_dist_clusters = min(15, len(all_features) // 30 + 1)
            self.minutiae_distribution_clusters = KMeans(n_clusters=n_dist_clusters, random_state=42, n_init=10)
            dist_features = [[f[0], f[1]] + f[6:10] for f in all_features]
            self.minutiae_distribution_clusters.fit(dist_features)
            
            # Update records with cluster assignments (FIXED: Convert to Python int)
            ridge_labels = [int(label) for label in self.ridge_flow_clusters.labels_]  # Convert to Python int
            dist_labels = [int(label) for label in self.minutiae_distribution_clusters.labels_]    # Convert to Python int
            
            for i, fp_id in enumerate(fingerprint_ids):
                index_collection.update_one(
                    {'fingerprint_id': fp_id},
                    {'$set': {
                        'ridge_cluster': ridge_labels[i],      # Now Python int
                        'distribution_cluster': dist_labels[i]  # Now Python int
                    }}
                )
        
        self.index_built = True
        print(f"✅ Index built for {len(all_features)} fingerprints")
        
        # Save clustering models
        self._save_models()


    
    def _save_models(self):
        """Save clustering models to file"""
        models = {
            'ridge_flow_clusters': self.ridge_flow_clusters,
            'minutiae_distribution_clusters': self.minutiae_distribution_clusters
        }
        with open('fingerprint_index_models.pkl', 'wb') as f:
            pickle.dump(models, f)
    
    def _load_models(self):
        """Load clustering models from file"""
        try:
            with open('fingerprint_index_models.pkl', 'rb') as f:
                models = pickle.load(f)
                self.ridge_flow_clusters = models['ridge_flow_clusters']
                self.minutiae_distribution_clusters = models['minutiae_distribution_clusters']
                self.index_built = True
                return True
        except FileNotFoundError:
            return False

    
    def get_candidates(self, query_minutiae_data, max_candidates=100):
        """
        Get candidate fingerprints using coarse indexing
        """
        if not self.index_built:
            if not self._load_models():
                print("⚠️ No index found. Building new index...")
                self.build_index()
        
        # Extract features from query
        query_features = self.extract_global_features(query_minutiae_data)
        if not query_features:
            return []
        
        # 1. Ridge flow based filtering
        if self.ridge_flow_clusters:
            ridge_feature = [query_features['dominant_orientation'], query_features['orientation_coherence']]
            ridge_cluster = int(self.ridge_flow_clusters.predict([ridge_feature])[0])  # Convert to int
            
            ridge_candidates = set()
            for cluster_id in [ridge_cluster]:
                ridge_records = index_collection.find({'ridge_cluster': cluster_id})  # Now uses Python int
                ridge_candidates.update(r['fingerprint_id'] for r in ridge_records)
        else:
            ridge_candidates = set(r['fingerprint_id'] for r in index_collection.find())
        
        # 2. Minutiae distribution based filtering
        if self.minutiae_distribution_clusters:
            dist_feature = ([query_features['total_minutiae'], query_features['termination_ratio']] + 
                        query_features['quad_density'])
            dist_cluster = int(self.minutiae_distribution_clusters.predict([dist_feature])[0])  # Convert to int
            
            dist_candidates = set()
            for cluster_id in [dist_cluster]:
                dist_records = index_collection.find({'distribution_cluster': cluster_id})  # Now uses Python int
                dist_candidates.update(r['fingerprint_id'] for r in dist_records)
        else:
            dist_candidates = set(r['fingerprint_id'] for r in index_collection.find())
        
        # 3. Hash bucket based filtering (unchanged)
        query_hashes = set(self.compute_minutiae_hash(query_minutiae_data))
        hash_candidates = set()
        
        if query_hashes:
            for hash_val in query_hashes:
                hash_records = index_collection.find({'hash_buckets': hash_val})
                hash_candidates.update(r['fingerprint_id'] for r in hash_records)
        else:
            hash_candidates = set(r['fingerprint_id'] for r in index_collection.find())
        
        # 4. Combine filtering results
        combined_candidates = (ridge_candidates & dist_candidates) | \
                            (hash_candidates & (ridge_candidates | dist_candidates))
        
        if not combined_candidates:
            combined_candidates = ridge_candidates | dist_candidates | hash_candidates
        
        # 5. Rank candidates by feature similarity (unchanged)
        candidate_scores = []
        for fp_id in combined_candidates:
            index_record = index_collection.find_one({'fingerprint_id': fp_id})
            if index_record and 'global_features' in index_record:
                db_features = index_record['global_features']
                
                # Compute simple similarity score
                score = 0
                # Minutiae count similarity
                count_diff = abs(query_features['total_minutiae'] - db_features['total_minutiae'])
                score += max(0, 1 - count_diff / 50)
                
                # Orientation similarity
                angle_diff = abs(query_features['dominant_orientation'] - db_features['dominant_orientation'])
                angle_diff = min(angle_diff, 2*np.pi - angle_diff)
                score += max(0, 1 - angle_diff / np.pi)
                
                # Density similarity
                density_diff = sum(abs(a - b) for a, b in zip(
                    query_features['quad_density'], db_features['quad_density']))
                score += max(0, 1 - density_diff)
                
                candidate_scores.append((fp_id, score, index_record['filename']))
        
        # Sort by score and return top candidates
        candidate_scores.sort(key=lambda x: x[1], reverse=True)
        
        result = []
        for fp_id, score, filename in candidate_scores[:max_candidates]:
            result.append({
                'fingerprint_id': fp_id,
                'filename': filename,
                'coarse_score': score
            })
        
        return result    




    

print("FingerprintCoarseIndex class defined successfully!")

FingerprintCoarseIndex class defined successfully!


In [17]:
# MCC Matching Functions

def _build_descriptors(minutiae, R=16, radial_bins=4, angular_bins=8, orient_bins=5):
    """Build MCC descriptors for minutiae"""
    processed_minutiae = []
    for m in minutiae:
        x, y = m[0], m[1]
        if len(m) > 2 and m[2] is not None:
            if isinstance(m[2], list):
                angle = m[2][0] if m[2] and m[2][0] is not None else 0.0
            else:
                angle = float(m[2])
        else:
            angle = 0.0
        processed_minutiae.append((x, y, angle))
    
    minutiae = processed_minutiae
    r_step = R / radial_bins
    a_step = 2*math.pi / angular_bins
    o_step = 2*math.pi / orient_bins

    descs = []
    for i, (xi, yi, thetai) in enumerate(minutiae):
        cyl = np.zeros((radial_bins, angular_bins, orient_bins), dtype=np.float32)

        for j, (xj, yj, thetaj) in enumerate(minutiae):
            if i == j:
                continue

            dx, dy = xj - xi, yj - yi
            r = math.hypot(dx, dy)
            if r > R:
                continue

            rb = min(int(r / r_step), radial_bins - 1)
            ang = (math.atan2(dy, dx) - thetai) % (2*math.pi)
            ab = min(int(ang / a_step), angular_bins - 1)
            d_theta = (thetaj - thetai + math.pi) % (2*math.pi)
            ob = min(int(d_theta / o_step), orient_bins - 1)

            cyl[rb, ab, ob] += 1.0

        v = cyl.flatten()
        v /= np.linalg.norm(v) + 1e-6
        descs.append(v)

    return np.vstack(descs) if descs else np.empty((0, radial_bins*angular_bins*orient_bins))

def _local_similarity(descA, descB):
    """Cosine similarity between descriptor pairs"""
    return np.clip(descA @ descB.T, 0.0, 1.0)

def mcc_match(minutiae_A, minutiae_B, R=16, radial_bins=4, angular_bins=8, orient_bins=5, topk=25):
    """Compute MCC similarity score between two fingerprints"""
    if not minutiae_A or not minutiae_B:
        return 0.0

    descA = _build_descriptors(minutiae_A, R, radial_bins, angular_bins, orient_bins)
    descB = _build_descriptors(minutiae_B, R, radial_bins, angular_bins, orient_bins)

    if descA.size == 0 or descB.size == 0:
        return 0.0

    S = _local_similarity(descA, descB)

    if topk is not None and S.size > topk:
        thresh = np.partition(S.flatten(), -topk)[-topk]
        S[S < thresh] = 0.0

    cost = 1.0 - S
    row_idx, col_idx = linear_sum_assignment(cost)

    matched_scores = S[row_idx, col_idx]
    if matched_scores.size == 0:
        return 0.0

    return float(np.mean(matched_scores))

print("MCC matching functions defined successfully!")

MCC matching functions defined successfully!


In [18]:
# Enhanced test function with coarse indexing
def test_fingerprint_match_with_indexing(input_image_path, threshold=0.3, params=None, use_indexing=True):
    """
    Enhanced fingerprint matching with coarse indexing
    """
    if params is None:
        params = {
            'spuriousMinutiaeThresh': 10,
            'invertImage': False
        }
    
    # Initialize indexing system
    coarse_index = FingerprintCoarseIndex()
    
    try:
        # Step 1: Read and process input image
        print(f" Reading image: {input_image_path}")
        img = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return {'success': False, 'message': f"Failed to read image from {input_image_path}"}
        
        print(" Enhancing fingerprint...")
        enhanced = fingerprint_enhancer.enhance_fingerprint(img)
        enhanced_uint8 = (enhanced * 255).astype(np.uint8)
        
        print(" Extracting minutiae features...")
        FeaturesTerm, FeaturesBif = extract_minutiae_features(
            enhanced_uint8,
            spuriousMinutiaeThresh=params.get('spuriousMinutiaeThresh', 10),
            invertImage=params.get('invertImage', False),
            showResult=False,
            saveResult=False
        )
        
        print(f"✔️ Found {len(FeaturesTerm)} terminations, {len(FeaturesBif)} bifurcations")
        
        # Convert to our format
        def minutiae_to_dict_local(features):
            return [{
                'x': int(f.locX),
                'y': int(f.locY),
                'angle': [float(a) if not math.isnan(a) else None for a in f.Orientation],
                'type': str(f.Type)
            } for f in features]
        
        query_data = {
            'terminations': minutiae_to_dict_local(FeaturesTerm),
            'bifurcations': minutiae_to_dict_local(FeaturesBif)
        }
        
        # Convert minutiae for matching
        input_minutiae = []
        for f in FeaturesTerm:
            angle = f.Orientation[0] if f.Orientation and len(f.Orientation) > 0 else 0.0
            input_minutiae.append((int(f.locX), int(f.locY), angle))
        for f in FeaturesBif:
            angle = f.Orientation[0] if f.Orientation and len(f.Orientation) > 0 else 0.0
            input_minutiae.append((int(f.locX), int(f.locY), angle))
        
        if not input_minutiae:
            return {'success': False, 'message': "No minutiae found in input image"}
        
        # Step 2: Use coarse indexing to get candidates
        if use_indexing:
            print(" Getting candidates using coarse indexing...")
            candidates = coarse_index.get_candidates(query_data, max_candidates=50)
            print(f" Found {len(candidates)} candidates from coarse indexing")
            
            if not candidates:
                return {
                    'success': True,
                    'max_score': 0.0,
                    'matched_filename': None,
                    'is_match': False,
                    'candidates_checked': 0,
                    'message': "No candidates found through coarse indexing"
                }
                
            # Get full records for candidates
            candidate_records = []
            for candidate in candidates:
                record = collection.find_one({'_id': candidate['fingerprint_id']})
                if record:
                    candidate_records.append(record)
        else:
            print(" Performing exhaustive search...")
            candidate_records = list(collection.find())
        
        # Step 3: Detailed matching on candidates only
        print(f" Performing detailed matching on {len(candidate_records)} candidates...")
        max_score = 0.0
        matched_filename = None
        
        for record in candidate_records:
            # Extract minutiae from database record
            db_minutiae = []
            
            for m in record.get('terminations', []):
                angle = m['angle'][0] if m['angle'] and len(m['angle']) > 0 and m['angle'][0] is not None else 0.0
                db_minutiae.append((m['x'], m['y'], angle))
            
            for m in record.get('bifurcations', []):
                angle = m['angle'][0] if m['angle'] and len(m['angle']) > 0 and m['angle'][0] is not None else 0.0
                db_minutiae.append((m['x'], m['y'], angle))
            
            if not db_minutiae:
                continue
            
            # Calculate similarity score using MCC
            score = mcc_match(input_minutiae, db_minutiae)
            
            if score > max_score:
                max_score = score
                matched_filename = record.get('filename', 'unknown')
        
        is_match = max_score >= threshold
        
        result = {
            'success': True,
            'max_score': max_score,
            'matched_filename': matched_filename,
            'is_match': is_match,
            'extracted_minutiae_count': len(input_minutiae),
            'candidates_checked': len(candidate_records),
            'total_in_db': collection.count_documents({}),
            'speedup_factor': collection.count_documents({}) / len(candidate_records) if candidate_records else 1,
            'message': f"{'Match found' if is_match else 'No match found'} - Best score: {max_score:.4f}"
        }
        
        print(f" {result['message']}")
        print(f" Checked {result['candidates_checked']}/{result['total_in_db']} fingerprints")
        print(f" Speedup: {result['speedup_factor']:.1f}x")
        
        if is_match:
            print(f" Matched with: {matched_filename}")
        
        return result
        
    except Exception as e:
        return {'success': False, 'message': f"Error during processing: {str(e)}"}

print("Enhanced test function defined successfully!")

Enhanced test function defined successfully!


In [19]:
# Enhanced usage function
def run_enhanced_fingerprint_test(image_path, similarity_threshold=0.3, use_indexing=True, rebuild_index=False):
    """
    Run fingerprint test with coarse indexing
    """
    if rebuild_index:
        print("🔨 Rebuilding coarse index...")
        coarse_index = FingerprintCoarseIndex()
        coarse_index.build_index()
    
    extraction_params = {
        'spuriousMinutiaeThresh': 10,
        'invertImage': False
    }
    
    result = test_fingerprint_match_with_indexing(
        input_image_path=image_path,
        threshold=similarity_threshold,
        params=extraction_params,
        use_indexing=use_indexing
    )
    
    if result['success']:
        print("\n" + "="*60)
        print("ENHANCED FINGERPRINT MATCHING RESULTS")
        print("="*60)
        print(f"Input Image: {image_path}")
        print(f"Extracted Minutiae: {result['extracted_minutiae_count']}")
        print(f"Candidates Checked: {result['candidates_checked']}/{result['total_in_db']}")
        print(f"Speedup Factor: {result['speedup_factor']:.1f}x")
        print(f"Best Match Score: {result['max_score']:.4f}")
        print(f"Threshold: {similarity_threshold}")
        print(f"Match Status: {'✅ MATCH' if result['is_match'] else '❌ NO MATCH'}")
        if result['is_match']:
            print(f"Matched Filename: {result['matched_filename']}")
        print("="*60)
    else:
        print(f"❌ Error: {result['message']}")
    
    return result

print("Enhanced usage function defined successfully!")

Enhanced usage function defined successfully!


## Usage Examples

In [20]:
# Example 1: Build the coarse index (run once)
print("Building coarse index for the first time...")
coarse_index = FingerprintCoarseIndex()
coarse_index.build_index()

Building coarse index for the first time...
🔨 Building coarse index...
✅ Index built for 803 fingerprints


In [21]:
# Example 2: Test with coarse indexing (fast)
test_image_path = r"C:\Users\golut\OneDrive\Documents\Projects\Test demo 2\Fingerprint-Feature-Extraction\jpg_images\1_5.jpg"

# Test with indexing enabled
result_fast = run_enhanced_fingerprint_test(
    image_path=test_image_path, 
    similarity_threshold=0.3, 
    use_indexing=True
)

print(f"\nResult: {result_fast}")

 Reading image: C:\Users\golut\OneDrive\Documents\Projects\Test demo 2\Fingerprint-Feature-Extraction\jpg_images\1_5.jpg
 Enhancing fingerprint...
 Extracting minutiae features...
✔️ Found 26 terminations, 13 bifurcations
 Getting candidates using coarse indexing...
 Found 50 candidates from coarse indexing
 Performing detailed matching on 50 candidates...
 Match found - Best score: 0.3846
 Checked 50/803 fingerprints
 Speedup: 16.1x
 Matched with: 1_5.jpg

ENHANCED FINGERPRINT MATCHING RESULTS
Input Image: C:\Users\golut\OneDrive\Documents\Projects\Test demo 2\Fingerprint-Feature-Extraction\jpg_images\1_5.jpg
Extracted Minutiae: 39
Candidates Checked: 50/803
Speedup Factor: 16.1x
Best Match Score: 0.3846
Threshold: 0.3
Match Status: ✅ MATCH
Matched Filename: 1_5.jpg

Result: {'success': True, 'max_score': 0.3846147060394287, 'matched_filename': '1_5.jpg', 'is_match': True, 'extracted_minutiae_count': 39, 'candidates_checked': 50, 'total_in_db': 803, 'speedup_factor': 16.06, 'message':

In [26]:
# Example 3: Compare performance - with vs without indexing
import time

test_image_path = r"C:\Users\golut\OneDrive\Documents\Projects\Test demo 2\Fingerprint-Feature-Extraction\jpg_images\1_5.jpg"

# Test WITH indexing
start_time = time.time()
result_indexed = run_enhanced_fingerprint_test(test_image_path, use_indexing=True)
indexed_time = time.time() - start_time

# Test WITHOUT indexing (exhaustive search)
start_time = time.time()
result_exhaustive = run_enhanced_fingerprint_test(test_image_path, use_indexing=False)
exhaustive_time = time.time() - start_time

print(f"\nPerformance Comparison:")
print(f"With Indexing: {indexed_time:.2f} seconds")
print(f"Without Indexing: {exhaustive_time:.2f} seconds")
print(f"Speedup: {exhaustive_time/indexed_time:.1f}x faster")

 Reading image: C:\Users\golut\OneDrive\Documents\Projects\Test demo 2\Fingerprint-Feature-Extraction\jpg_images\1_5.jpg
 Enhancing fingerprint...
 Extracting minutiae features...
✔️ Found 26 terminations, 13 bifurcations
 Getting candidates using coarse indexing...
 Found 50 candidates from coarse indexing
 Performing detailed matching on 50 candidates...
 Match found - Best score: 0.3846
 Checked 50/803 fingerprints
 Speedup: 16.1x
 Matched with: 1_5.jpg

ENHANCED FINGERPRINT MATCHING RESULTS
Input Image: C:\Users\golut\OneDrive\Documents\Projects\Test demo 2\Fingerprint-Feature-Extraction\jpg_images\1_5.jpg
Extracted Minutiae: 39
Candidates Checked: 50/803
Speedup Factor: 16.1x
Best Match Score: 0.3846
Threshold: 0.3
Match Status: ✅ MATCH
Matched Filename: 1_5.jpg
 Reading image: C:\Users\golut\OneDrive\Documents\Projects\Test demo 2\Fingerprint-Feature-Extraction\jpg_images\1_5.jpg
 Enhancing fingerprint...
 Extracting minutiae features...
✔️ Found 26 terminations, 13 bifurcations


In [23]:
# Example 4: Test multiple images
test_images = [
    "path/to/test1.jpg",
    "path/to/test2.jpg",
    "path/to/test3.jpg"
]

results = []
for img_path in test_images:
    print(f"\nTesting: {img_path}")
    result = run_enhanced_fingerprint_test(img_path, use_indexing=True)
    results.append({
        'image': img_path,
        'match_found': result.get('is_match', False),
        'score': result.get('max_score', 0.0),
        'matched_file': result.get('matched_filename', 'None')
    })

print("\nSummary of all tests:")
for r in results:
    status = "✅ MATCH" if r['match_found'] else "❌ NO MATCH"
    print(f"{r['image']}: {status} (Score: {r['score']:.4f}) -> {r['matched_file']}")


Testing: path/to/test1.jpg
 Reading image: path/to/test1.jpg
❌ Error: Failed to read image from path/to/test1.jpg

Testing: path/to/test2.jpg
 Reading image: path/to/test2.jpg
❌ Error: Failed to read image from path/to/test2.jpg

Testing: path/to/test3.jpg
 Reading image: path/to/test3.jpg
❌ Error: Failed to read image from path/to/test3.jpg

Summary of all tests:
path/to/test1.jpg: ❌ NO MATCH (Score: 0.0000) -> None
path/to/test2.jpg: ❌ NO MATCH (Score: 0.0000) -> None
path/to/test3.jpg: ❌ NO MATCH (Score: 0.0000) -> None


In [24]:
# Example 5: Rebuild index when database changes
print("Rebuilding index after database updates...")
result = run_enhanced_fingerprint_test(
    "path/to/test.jpg", 
    rebuild_index=True,  # This will rebuild the index
    use_indexing=True
)

Rebuilding index after database updates...
🔨 Rebuilding coarse index...
🔨 Building coarse index...
✅ Index built for 803 fingerprints
 Reading image: path/to/test.jpg
❌ Error: Failed to read image from path/to/test.jpg


## System Information and Performance

In [27]:
# Check system status
print("System Status:")
print("=" * 40)
print(f"Total Fingerprints in DB: {collection.count_documents({})}")
print(f"Index Records: {index_collection.count_documents({})}")

# Check if index models exist
import os
if os.path.exists('fingerprint_index_models.pkl'):
    print("✅ Index models file exists")
else:
    print("❌ Index models file not found - run build_index() first")

print("\nSystem ready for fingerprint matching!")
print("Use run_enhanced_fingerprint_test('your_image.jpg') to test")

System Status:
Total Fingerprints in DB: 803
Index Records: 803
✅ Index models file exists

System ready for fingerprint matching!
Use run_enhanced_fingerprint_test('your_image.jpg') to test
